In [ ]:
!git clone https://github.com/penfever/rtfm
!pip install uv

Cloning into 'rtfm'...
remote: Enumerating objects: 709, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 709 (delta 156), reused 151 (delta 128), pack-reused 505 (from 1)
Receiving objects: 100% (709/709), 19.09 MiB | 6.60 MiB/s, done.
Resolving deltas: 100% (434/434), done.
Updating files: 100% (178/178), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 32.7 MB/s eta 0:00:00


In [ ]:
%cd /content/rtfm

/content/rtfm


In [ ]:
!sed -i 's/catboost==1.1.1/catboost/g; s/pandas/pandas>=2.0.0/g' /content/rtfm/requirements.txt

In [ ]:
!uv pip install -r requirements.txt && uv pip install --no-deps git+https://github.com/mlfoundations/tableshift.git

Streaming output truncated to the last 5000 lines.
catboost   ------------------------------ 14.38 MiB/93.61 MiB
nvidia-cusolver-cu12 ------------------------------ 14.53 MiB/122.01 MiB
nvidia-cusparse-cu12 ------------------------------ 14.51 MiB/197.84 MiB
nvidia-cufft-cu12 ------------------------------ 14.43 MiB/201.66 MiB
nvidia-cublas-cu12 ------------------------------ 14.37 MiB/346.60 MiB
⠙ Preparing packages... (58/72)
jedi       ------------------------------ 1.07 MiB/1.50 MiB
botocore   ------------------------------ 9.50 MiB/11.89 MiB
nvidia-nvjitlink-cu12 ------------------------------ 14.28 MiB/20.09 MiB
nvidia-cuda-nvrtc-cu12 ------------------------------ 14.39 MiB/23.50 MiB
gradio     ------------------------------ 12.70 MiB/44.04 MiB
nvidia-curand-cu12 ------------------------------ 14.65 MiB/53.70 MiB
ray        ------------------------------ 12.74 MiB/64.92 MiB
bitsandbytes ------------------------------ 14.49 MiB/72.54 MiB
catboost   ------------------------------ 

In [ ]:
!uv pip install git+https://github.com/jpgard/llama-recipes.git
!uv pip install -e .
!uv pip install git+https://github.com/penfever/tabliblib.git

Using Python 3.11.11 environment at: /usr
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating https://github.com/jpgard/llama-recipes.git (HEAD)
   Updating h

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import nltk
nltk.download('punkt_tab')
import os
import logging
import sys
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
import pandas as pd
import torch
from transformers import AutoTokenizer, LlamaForCausalLM, AutoConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

from rtfm.configs import TrainConfig, TokenizerConfig
from rtfm.inference_utils import InferenceModel
from rtfm.serialization.serializers import get_serializer
from rtfm.tokenization.text import prepare_tokenizer

In [ ]:
train_config = TrainConfig(model_name="mlfoundations/tabula-8b", context_length=8192)

# If using a base llama model (not fine-tuned TabuLa),
# make sure to set add_serializer_tokens=False
# (because we do not want to use special tokens for
# the base model which is not trained on them).
tokenizer_config = TokenizerConfig()

# Load the configuration
config = AutoConfig.from_pretrained(train_config.model_name)

# Set the torch_dtype to bfloat16 which matches TabuLa train/eval setup
config.torch_dtype = 'bfloat16'

device = "cuda" if torch.cuda.is_available() else "cpu"

model = LlamaForCausalLM.from_pretrained(
    train_config.model_name, device_map="auto", config=config).to(device)

tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
serializer = get_serializer(train_config.serializer_cls)

tokenizer, model = prepare_tokenizer(
    model,
    tokenizer=tokenizer,
    pretrained_model_name_or_path=train_config.model_name,
    model_max_length=train_config.context_length,
    use_fast_tokenizer=tokenizer_config.use_fast_tokenizer,
    serializer_tokens_embed_fn=tokenizer_config.serializer_tokens_embed_fn,
    serializer_tokens=serializer.special_tokens
    if tokenizer_config.add_serializer_tokens
    else None,
)

inference_model = InferenceModel(model=model, tokenizer=tokenizer, serializer=serializer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

In [ ]:
# Define the OpenML datasets to use
DATASETS = {
    'har': 1478,
    'breast_cancer': 15,
    'semeion': 1501,
    'airlines': 42742
}

In [ ]:
for dataset_name, dataset_id in DATASETS.items():
    print(f"\n{'='*50}\nFetching dataset: {dataset_name} (ID: {dataset_id})\n{'='*50}")

    # Fetch the dataset
    data = fetch_openml(data_id=dataset_id, as_frame=True)

    # Extract features and target
    X = data.data
    y = data.target
    # Get the column name if y is a Series
    if isinstance(y, pd.Series):
        y_colname = y.name
    else:
        # If it's a DataFrame, get the first column name
        y_colname = y.columns[0]

    # Create train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    # Create a subset of 8 examples as labeled examples
    labeled_indices = np.random.choice(X_train.shape[0], size=8, replace=False)
    labeled_examples = pd.concat([
        X_train.iloc[labeled_indices].reset_index(drop=True),
        y_train.iloc[labeled_indices].reset_index(drop=True)
    ], axis=1)

    # Initialize lists to store results and metrics
    all_results = []
    y_true = []
    y_pred_proba = {}
    y_pred = []

    # Process each test example
    for i in range(len(X_test)):
        target_example = X_test.iloc[i]

        # Get the model prediction
        output = inference_model.predict(
            target_example=target_example,
            target_colname=y_colname,
            target_choices=[str(y_val) for y_val in y.unique()],
            labeled_examples=labeled_examples,
        )

        # Store results
        all_results.append(output)
        y_true.append(y_test.iloc[i][y_colname])
        y_pred.append(output['prediction'])

        # Store probabilities if available
        if 'probabilities' in output:
            for cls, prob in output['probabilities'].items():
                if cls not in y_pred_proba:
                    y_pred_proba[cls] = []
                y_pred_proba[cls].append(prob)

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

    # Calculate ROC AUC if applicable (for binary classification)
    if len(y.unique()) == 2 and all(cls in y_pred_proba for cls in y.unique()):
        pos_class = str(y.unique()[1])
        roc_auc = roc_auc_score(
            [1 if y == pos_class else 0 for y in y_true],
            y_pred_proba[pos_class]
        )
        print(f"ROC AUC: {roc_auc:.4f}")

    # Save results for this dataset
    dataset_results = {
        'dataset_name': dataset_name,
        'dataset_id': dataset_id,
        'accuracy': accuracy,
        'all_predictions': all_results,
        'y_true': y_true,
        'y_pred': y_pred
    }

    if 'roc_auc' in locals():
        dataset_results['roc_auc'] = roc_auc


Fetching dataset: har (ID: 1478)


NameError: name 'inference_model' is not defined